In [1]:
from week2_novel import *
import nbimporter, importlib
import utility_fncs, global_vars

In [2]:
import inspect
import json
from typing import get_origin, get_args, Literal

In [3]:
def create_tool_from_function(func):
    """
    Generates an OpenAI-compatible tool definition from a Python function.

    This function leverages type hints and a structured docstring to create 
    the JSON schema required by the OpenAI API for tool-calling.

    It supports multi-line function descriptions and multi-line parameter
    descriptions.

    Args:
        func (callable): The function to be converted into a tool.

    Returns:
        dict: A dictionary representing the tool in the format expected by
              the OpenAI API.
    """
    # Get the clean, un-indented docstring
    full_docstring = inspect.getdoc(func)
    if not full_docstring:
        raise ValueError("The function must have a docstring to be used as a tool.")
    
    lines = [line.strip() for line in full_docstring.strip().split('\n')]
    
    # --- 1. Parse Function and Parameter Descriptions ---
    try:
        args_section_index = lines.index('Args:')
    except ValueError:
        args_section_index = len(lines)

    func_description = "\n".join(lines[:args_section_index]).strip()
    
    # Robustly parse multi-line argument descriptions
    param_docs = {}
    current_param_name = None
    args_section_lines = lines[args_section_index + 1:]

    for line in args_section_lines:
        if ':' in line:
            param_name, param_desc = line.split(':', 1)
            param_name = param_name.split('(')[0].strip()
            # Start a new list of description lines for this parameter
            param_docs[param_name] = [param_desc.strip()]
            current_param_name = param_name
        elif current_param_name:
            # If a line doesn't have a colon, it's a continuation of the last parameter
            param_docs[current_param_name].append(line.strip())

    # Join the multi-line descriptions back into single strings
    for name, desc_lines in param_docs.items():
        param_docs[name] = "\n".join(desc_lines)

    # --- 2. Introspect Function Signature ---
    sig = inspect.signature(func)
    parameters_schema = {"type": "object", "properties": {}, "required": []}
    type_mapping = {str: "string", int: "integer", float: "number", bool: "boolean", dict: "object"}

    for name, param in sig.parameters.items():
        if param.default == inspect.Parameter.empty and name != 'self':
            parameters_schema["required"].append(name)

        param_info = {}
        if get_origin(param.annotation) is Literal:
            param_info["type"] = "string"
            param_info["enum"] = list(get_args(param.annotation))
        else:
            param_info["type"] = type_mapping.get(param.annotation, "string")
            
        if name in param_docs:
            param_info["description"] = param_docs[name]

        parameters_schema["properties"][name] = param_info
        
    if not parameters_schema["properties"]:
        parameters_schema = {"type": "object", "properties": {}}

    # --- 3. Assemble Final Tool ---
    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": func_description,
            "parameters": parameters_schema,
        },
    }

In [4]:
# tool functions
# Define function with type hints and a specific docstring format

def get_current_weather(location: str, unit: Literal["celsius", "fahrenheit"] = "celsius"):
    """Gets the current weather in a given location.
    It can be a city, state format

    Args:
        location (str): The city and state, e.g., San Francisco, CA.
        a example if of form San Francisco, CA
        unit (str): The unit of temperature, either 'celsius' or 'fahrenheit'.
    """
    # function logic     
    return_dict={
        'type':'json',
        'content':'',
    }
    if "san francisco" in location.lower():
        return_dict['content']=json.dumps({"location": location, "temperature": "15", "unit": unit})
    elif "tokyo" in location.lower():
        return_dict['content']= json.dumps({"location": location, "temperature": "22", "unit": unit})
    else:
        return_dict['content']= json.dumps({"location": location, "temperature": "unknown"})
    return return_dict


def get_stock_price(symbol: str):
    """Retrieves the current stock price for a given ticker symbol.
    
    Args:
        symbol (str): The stock ticker symbol, e.g., AAPL for Apple, GOOG for Google.
    """
    return_dict={
        'type':'json',
        'content':'',
    }
    if symbol.upper() == "AAPL":
        return_dict['content']= json.dumps({"symbol": "AAPL", "price": "175.28"})
    else:
        return_dict['content']= json.dumps({"symbol": symbol, "price": "not found"})
    return return_dict



In [5]:
import base64
from io import BytesIO
from PIL import Image

def create_image(msg: str):
    """"Generates an image from a detailed text prompt. Use this tool when a user explicitly asks to draw or create a picture,
    OR when a visual aid would significantly help in explaining a complex concept or answering a question."

    Args:
        msg(str) : the string that can be passed as a input to openai image generation (openai.image.generate) call's  keyword argument prompt. 
        It has to obtained from the user's message or intention requesting for a iamge
    """
    return_dict={
        'type':'image',
        'content':'',
    }
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=msg,
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return_dict['content']=Image.open(BytesIO(image_data))
    print(return_dict['content'])
    return return_dict

In [6]:
#openai
#create_image('create image of a sunrise in the Himalayas')

In [7]:
from openai import OpenAI
tool_fncs = []
tool_fncs.append(create_tool_from_function(get_current_weather))
available_functions = {
        "get_current_weather": get_current_weather,
        "get_stock_price": get_stock_price,
        "create_image": create_image,
    }
def chat_gpt_tool_call(qt, file=None, model=None):
    #print(qt)
    #msg = utility_fncs.create_gpt_prompt(qt,"user")
    #print(msg)
    #global_vars.prompt_gpt = utility_fncs.append_conv(global_vars.prompt_gpt,msg)
    #print(global_vars.prompt_gpt)
    user_image = None
    global_vars.prompt_gpt.append({"role": "user", "content":qt})
    openai = OpenAI()
    response = openai.chat.completions.create(
        model=global_vars.model_openai_4_5_preview,
        messages=global_vars.prompt_gpt,        
        tools=tool_fncs
    )
    #print(response)
    reply = response.choices[0].message
    reply_tool_calls = reply.tool_calls
    if reply_tool_calls:
        # list to hold the reply from tool calls
        tool_messages = []
        for reply_tool_call in reply_tool_calls:
            function_name = reply_tool_call.function.name
            function_to_call = available_functions.get(function_name,None)
            if not function_to_call:
                continue
            function_args = json.loads(reply_tool_call.function.arguments)

            # call our tool
            function_response  = function_to_call(**function_args)
            if (isinstance(function_response,dict)):
                if function_response.get('type')=='image':
                    user_image = function_response.get('content')
                    function_response = "Image created as per user's request"
                else:
                    function_response=function_response.get('content','')
            tool_messages.append(
                {
                    "tool_call_id" : reply_tool_call.id,
                    "role" : "tool",
                    "name" : function_name,
                    "content" : function_response,
                }
            )
            global_vars.prompt_gpt.append(reply)
            #global_vars.prompt_gpt = utility_fncs.append_conv(global_vars.prompt_gpt, reply)
            #global_vars.prompt_gpt = utility_fncs.append_conv(global_vars.prompt_gpt, *tool_messages)
            global_vars.prompt_gpt.append(*tool_messages)
            response = openai.chat.completions.create(
                model=model_openai_4onano,
                messages=global_vars.prompt_gpt,
            )
    print(user_image)
    return response.choices[0].message.content, user_image
    
                

        
        

In [8]:
#global_vars.prompt_gpt = utility_fncs.reset_conv_history(global_vars.prompt_gpt,utility_fncs.create_gpt_prompt(global_vars.system_gpt_msg,'assistant'))
#chat_gpt_tool_call('weather in tokyo?')
#launcher = utility_fncs.get_gradio_launcher(chat_gpt_tool_call)

In [9]:
tool_fncs.append(create_tool_from_function(create_image))
global_vars.prompt_gpt = utility_fncs.reset_conv_history(global_vars.prompt_gpt,utility_fncs.create_gpt_prompt(global_vars.system_gpt_msg,'assistant'))
launcher = utility_fncs.get_gradio_multi_modal_launcher(chat_gpt_tool_call)

In [10]:
launcher.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [11]:
"""
{
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "tool_calls": [
          {
            "id": "call_abc123",
            "type": "function",
            "function": {
              "name": "get_current_weather",
              "arguments": "{\"location\": \"San Francisco, CA\"}"
            }
          },
          {
            "id": "call_xyz789",
            "type": "function",
            "function": {
              "name": "get_stock_price",
              "arguments": "{\"symbol\": \"AAPL\"}"
            }
          }
        ]
      }
    }
  ],
  ...
}
"""

'\n{\n  "choices": [\n    {\n      "finish_reason": "tool_calls",\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": null,\n        "tool_calls": [\n          {\n            "id": "call_abc123",\n            "type": "function",\n            "function": {\n              "name": "get_current_weather",\n              "arguments": "{"location": "San Francisco, CA"}"\n            }\n          },\n          {\n            "id": "call_xyz789",\n            "type": "function",\n            "function": {\n              "name": "get_stock_price",\n              "arguments": "{"symbol": "AAPL"}"\n            }\n          }\n        ]\n      }\n    }\n  ],\n  ...\n}\n'